In [2]:
import pandas as pd
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
from tqdm import tqdm
import argparse
import glob

In [4]:
import xml.etree.ElementTree as ET 
import cv2
import ast

In [ ]:
import numpy as np
import sys

In [5]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as immg

In [1]:






import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as immg

import random

import torch

import torchvision
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"



import albumentations as A
from albumentations.pytorch import ToTensorV2

ModuleNotFoundError: No module named 'albumentations'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path_an = "/content/drive/MyDrive/PCB/PCB_DATASET 1/Annotations"
print(path_an)

/content/drive/MyDrive/PCB/PCB_DATASET 1/Annotations


In [4]:
dataset = {
            "xmin":[],
            "ymin":[],   
            "xmax":[],
            "ymax":[],
            "class":[],    
            "file":[],
            "width":[],
            "height":[],
           }
all_files = []
for path, subdirs, files in os.walk(path_an):

    for name in files:
        all_files.append(os.path.join(path, name))
        
for anno in all_files:
    
    tree = ET.parse(anno)
    
    for elem in tree.iter():
        # print(elem)
        
        if 'size' in elem.tag:
            
            for attr in list(elem):
                if 'width' in attr.tag: 
                    width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    height = int(round(float(attr.text)))    

        if 'object' in elem.tag:
            
            for attr in list(elem):
                
                
                if 'name' in attr.tag:
                    name = attr.text                 
                    dataset['class']+=[name]
                    dataset['width']+=[width]
                    dataset['height']+=[height] 
                    dataset['file']+=[anno.split('/')[-1][0:-4]] 
                            
                if 'bndbox' in attr.tag:
                    for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            dataset['xmin']+=[xmin]
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            dataset['ymin']+=[ymin]                                
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            dataset['xmax']+=[xmax]                                
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            dataset['ymax']+=[ymax] 

In [5]:
data=pd.DataFrame(dataset)
data

,xmin,ymin,xmax,ymax,class,file,width,height
0,705,1114,771,1169,short,01_short_07,3034,1586
1,1645,524,1744,617,short,01_short_07,3034,1586
2,1599,1227,1688,1312,short,01_short_07,3034,1586
3,1105,1253,1166,1321,short,01_short_11,3034,1586
4,2121,927,2206,1026,short,01_short_11,3034,1586
...,...,...,...,...,...,...,...,...
2690,2197,323,2240,399,spurious_copper,12_spurious_copper_03,2529,2530
2691,907,1041,1006,1097,spurious_copper,12_spurious_copper_03,2529,2530
2692,2091,1221,2184,1261,spurious_copper,12_spurious_copper_03,2529,2530
2693,2358,1654,2397,1735,spurious_copper,12_spurious_copper_03,2529,2530


In [6]:
train, test = train_test_split(data, shuffle=True, test_size=0.2, random_state=34)

In [7]:
train.shape, test.shape

((2156, 8), (539, 8))

In [8]:
classes_la = {"missing_hole": 0, "mouse_bite": 1, "open_circuit":2, "short": 3, 'spur': 4,'spurious_copper':5}

train["class"] = train["class"].apply(lambda x: classes_la[x])
test["class"] = test["class"].apply(lambda x: classes_la[x])

In [9]:
train.head()

,xmin,ymin,xmax,ymax,class,file,width,height
714,1873,210,1941,265,2,07_open_circuit_07,2904,1921
2681,1395,935,1457,992,5,12_spurious_copper_10,2529,2530
777,995,428,1040,468,2,09_open_circuit_02,2775,2159
2153,1255,1155,1330,1209,1,11_mouse_bite_09,2282,2248
2284,2609,1020,2665,1235,5,04_spurious_copper_11,3056,2464


In [10]:
test.head()

,xmin,ymin,xmax,ymax,class,file,width,height
1393,1608,1519,1691,1570,4,06_spur_03,2868,2316
2567,1804,1839,1858,1926,5,11_spurious_copper_04,2282,2248
86,1853,1033,1955,1121,3,04_short_16,3056,2464
2627,1666,1859,1791,1908,5,12_spurious_copper_01,2529,2530
1093,717,970,778,1044,0,11_missing_hole_03,2282,2248


In [24]:
df = train.copy()

df_grp = df.groupby(['file'])

class fcbData(object):
    def __init__(self, df, IMG_DIR, transforms): 
        self.df = df
        self.img_dir = IMG_DIR
        self.image_ids = self.df['file'].unique().tolist()
        self.transforms = transforms
        
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        a = ''
        if "missing" in image_id.split('_'):
            a = 'Missing_hole/'
        elif "mouse" in image_id.split('_'):
            a = 'Mouse_bite/'
        elif "open" in image_id.split('_'):
            a = 'Open_circuit/'
        elif "short" in image_id.split('_'):
            a = 'Short/'
        elif "spur" in image_id.split('_'):
            a = 'Spur/'
        elif "spurious" in image_id.split('_'):
            a = 'Spurious_copper/'
        print("Debug:", self.img_dir, a, image_id)  # Debugging statement
        path = os.path.join(self.img_dir, a, image_id + ".jpg")
        print(path)
        image_values = self.df[self.df['file'] == image_id]
        image = cv2.imread(path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        boxes = image_values[['xmin', 'ymin', 'xmax', 'ymax']].to_numpy()
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        labels = image_values["class"].values
        labels = torch.tensor(labels)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([idx])
        target['area'] = torch.as_tensor(area, dtype=torch.float32)
        target['iscrowd'] = torch.zeros(len(classes_la), dtype=torch.int64)

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
        
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return torch.tensor(image), target, image_id

In [25]:
def get_train_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

print(df)
print(path)

      xmin  ymin  xmax  ymax  class                   file  width  height
714   1873   210  1941   265      2     07_open_circuit_07   2904    1921
2681  1395   935  1457   992      5  12_spurious_copper_10   2529    2530
777    995   428  1040   468      2     09_open_circuit_02   2775    2159
2153  1255  1155  1330  1209      1       11_mouse_bite_09   2282    2248
2284  2609  1020  2665  1235      5  04_spurious_copper_11   3056    2464
...    ...   ...   ...   ...    ...                    ...    ...     ...
1571  1168  1271  1227  1366      4             08_spur_05   2759    2154
324   1752  1644  1871  1790      3            10_short_04   2240    2016
1897  1439  1083  1533  1113      1       06_mouse_bite_08   2868    2316
2538  1436   433  1484   531      5  09_spurious_copper_07   2775    2159
1146   857   224   937   313      0     11_missing_hole_09   2282    2248

[2156 rows x 8 columns]
None


In [32]:
path = "/content/drive/MyDrive/PCB/PCB_DATASET 1/images"
fcb_dataset   = fcbData(df, path, get_train_transform())

In [33]:
df = train.copy()
df_grp = df.groupby(['file'])
image_ids = df['file'].unique()
valid_ids = image_ids[-665:]
train_ids = image_ids[:-665]
valid_df = df[df['file'].isin(valid_ids)]
train_df = df[df['file'].isin(train_ids)]
train_df.shape,valid_df.shape

((0, 8), (2156, 8))

In [34]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = fcbData(df, path, get_train_transform())
valid_dataset = fcbData(df, path, get_valid_transform())

indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=6,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=6,
    collate_fn=collate_fn
)

In [35]:
num_classes = 6

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [36]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0001, weight_decay=0.0005,)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

num_epochs = 1

In [37]:
best_epoch = 0
min_loss = sys.maxsize

for epoch in range(num_epochs):
    tk = tqdm(train_data_loader)
    model.train();
    for images, targets, image_ids in tk:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        tk.set_postfix(train_loss=loss_value)
    tk.close()
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()
    
    print(f"Epoch #{epoch} loss: {loss_value}") 
        
    #validation 
    model.eval();
    with torch.no_grad():
        tk = tqdm(valid_data_loader)
        for images, targets, image_ids in tk:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            val_output = model(images)
            val_output = [{k: v.to(device) for k, v in t.items()} for t in val_output]
            IOU = []
            for j in range(len(val_output)):
                a,b = val_output[j]['boxes'].cpu().detach(), targets[j]['boxes'].cpu().detach()
                chk = torchvision.ops.box_iou(a,b)
                res = np.nanmean(chk.sum(axis=1)/(chk>0).sum(axis=1))
                IOU.append(res)
            tk.set_postfix(IoU=np.mean(IOU))
        tk.close()

  0%|          | 0/623 [00:00<?, ?it/s]

Debug:Debug:Debug:Debug:  /content/drive/MyDrive/PCB/PCB_DATASET 1/imagesDebug:  /content/drive/MyDrive/PCB/PCB_DATASET 1/images /content/drive/MyDrive/PCB/PCB_DATASET 1/imagesMouse_bite//content/drive/MyDrive/PCB/PCB_DATASET 1/images  /content/drive/MyDrive/PCB/PCB_DATASET 1/imagesSpurious_copper/   Open_circuit/11_mouse_bite_09Spurious_copper/  Debug: 
 12_spurious_copper_10 04_spurious_copper_1107_open_circuit_07

/content/drive/MyDrive/PCB/PCB_DATASET 1/imagesOpen_circuit//content/drive/MyDrive/PCB/PCB_DATASET 1/images/Mouse_bite/11_mouse_bite_09.jpg/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Spurious_copper/04_spurious_copper_11.jpg
 
/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Spurious_copper/12_spurious_copper_10.jpg /content/drive/MyDrive/PCB/PCB_DATASET 1/images/Open_circuit/07_open_circuit_07.jpg

Short/
 12_short_0409_open_circuit_02

/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Short/12_short_04.jpg/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Open_circuit/09_o

  0%|          | 0/623 [00:00<?, ?it/s]

Debug:Debug: Debug: /content/drive/MyDrive/PCB/PCB_DATASET 1/imagesDebug: /content/drive/MyDrive/PCB/PCB_DATASET 1/images /content/drive/MyDrive/PCB/PCB_DATASET 1/imagesDebug:  Open_circuit/  /content/drive/MyDrive/PCB/PCB_DATASET 1/images/content/drive/MyDrive/PCB/PCB_DATASET 1/imagesOpen_circuit/ Spurious_copper/  Mouse_bite/ 07_open_circuit_07Spurious_copper/ 12_spurious_copper_10  09_open_circuit_02

11_mouse_bite_09

/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Spurious_copper/12_spurious_copper_10.jpg04_spurious_copper_11/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Open_circuit/07_open_circuit_07.jpg/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Open_circuit/09_open_circuit_02.jpg/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Mouse_bite/11_mouse_bite_09.jpg




/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Spurious_copper/04_spurious_copper_11.jpg
Debug: /content/drive/MyDrive/PCB/PCB_DATASET 1/images Short/ 12_short_04
/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Sh

In [38]:
img,target,_ = valid_dataset[3]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])[0]
    
print('predicted #boxes: ', len(prediction['boxes']))
print('real #boxes: ', len(target['boxes']))

Debug: /content/drive/MyDrive/PCB/PCB_DATASET 1/images Mouse_bite/ 11_mouse_bite_09
/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Mouse_bite/11_mouse_bite_09.jpg
predicted #boxes:  29
real #boxes:  5


In [39]:
y_true =[]
y_pred = []
for i in range(50):
    img,target,_ = valid_dataset[i]
    model.eval()
    with torch.no_grad():
        prediction = model([img.to(device)])[0]
        y_true.append(target['labels'][0])
        y_pred.append(prediction['labels'][0])

Debug: /content/drive/MyDrive/PCB/PCB_DATASET 1/images Open_circuit/ 07_open_circuit_07
/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Open_circuit/07_open_circuit_07.jpg
Debug: /content/drive/MyDrive/PCB/PCB_DATASET 1/images Spurious_copper/ 12_spurious_copper_10
/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Spurious_copper/12_spurious_copper_10.jpg
Debug: /content/drive/MyDrive/PCB/PCB_DATASET 1/images Open_circuit/ 09_open_circuit_02
/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Open_circuit/09_open_circuit_02.jpg
Debug: /content/drive/MyDrive/PCB/PCB_DATASET 1/images Mouse_bite/ 11_mouse_bite_09
/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Mouse_bite/11_mouse_bite_09.jpg
Debug: /content/drive/MyDrive/PCB/PCB_DATASET 1/images Spurious_copper/ 04_spurious_copper_11
/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Spurious_copper/04_spurious_copper_11.jpg
Debug: /content/drive/MyDrive/PCB/PCB_DATASET 1/images Short/ 12_short_04
/content/drive/MyDrive/PCB/PCB_DATASET 1/images/Sh

In [40]:
yy_pred = []
for v in y_pred:
    yy_pred.append(v.cpu())

In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_true, yy_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.62      1.00      0.76         8
           2       1.00      0.82      0.90        11
           3       1.00      1.00      1.00         9
           4       0.75      1.00      0.86         6
           5       0.91      0.91      0.91        11

    accuracy                           0.84        50
   macro avg       0.71      0.79      0.74        50
weighted avg       0.79      0.84      0.80        50



In [42]:
import pickle

# Save the model to a pickle file
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [49]:
torch.save(model.state_dict(), 'model.h5')